# Compare functional scores to natural sequence diversity

In [1]:
# Imports
import os
import warnings
import scipy as sp
import pandas as pd
import altair as alt

# Suppress warnings
warnings.simplefilter("ignore")

# Allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

In [2]:
# this cell is tagged as `parameters` for papermill parameterization
func_effects = None
min_times_seen = None
site_numbering_map = None

natural_variation = None

In [3]:
# # Uncomment for running interactive
# func_effects = "../results/func_effects/averages/HEK293T_entry_func_effects.csv"
# min_times_seen = 3
# site_numbering_map = "../data/site_numbering_map.csv"

# natural_variation = "../non-pipeline_analyses/RABV_nextstrain/Results/G_variation.csv"

Process functional effects by filtering min times seen and averaging all amino-acid mutations (exlcuding stop codons) and map to natural sites

In [4]:
# Load data and filter functional effects
func_effects = (
    pd.read_csv(func_effects)
    .query("times_seen >= @min_times_seen & mutant != '*' & mutant != '-'")
    .reset_index(drop=True)
)

# Load site map
site_map = (
    pd.read_csv(site_numbering_map)
    .rename(columns={
        "Unnamed: 0" : "natural_sequence_site",
        "reference_site" : "site",
    })
)

# Merge site map functional effects
func_effects = (
    func_effects.merge(
        site_map,
        how="left",
        on=["site"],
        validate="many_to_one",
    )
)

# Average site functional effects
func_effects = (
    func_effects.groupby(["site", "wildtype"])
    .aggregate({
        "effect" : "mean",
        "natural_sequence_site" : "first",
        "region" : "first",
    })
    .reset_index()
)

# Load natural variation data
natural_var = (
    pd.read_csv(natural_variation)
    .rename(columns={
        "site" : "natural_sequence_site",
    })
)

# Merge site map functional effects
func_effects = (
    func_effects.merge(
        natural_var[["entropy", "n_effective", "natural_sequence_site"]],
        how="left",
        on=["natural_sequence_site"],
        validate="one_to_one",
    )
)

In [5]:
subplots = []
# Can subset on specific regions but for now just show all
for index,region in enumerate(["all"]):

    # Subset data based on region
    subsetted_data = None
    if region == "all":
        subsetted_data = func_effects
    else:
        subsetted_data = func_effects.query("region == @region")

    # Calculate statistics
    r, p = sp.stats.pearsonr(
        subsetted_data.dropna()["effect"], 
        subsetted_data.dropna()["entropy"]
    )
    print(f"r correlation for {region}: {r:.2f}")
        
    curr_subplot = alt.Chart(subsetted_data).mark_point(
        filled=True, 
        color="black", 
        size=75,
        opacity=0.35,
    ).encode(
        alt.X(
            "entropy",
            axis=alt.Axis(
                title=["site entropy in", "natural sequences"], 
                values=[0,0.5,1],
                domainWidth=1,
                domainColor="black",
                tickColor="black",
            ),
            scale=alt.Scale(domain=[-0.05, 1.3])
        ),
        alt.Y(
            "effect",
            axis=alt.Axis(
                title=["site mean", "effect on cell entry"], 
                values=[-7,-6,-5,-4,-3,-2,-1,0],
                domainWidth=1,
                domainColor="black",
                tickColor="black",
            ),
            scale=alt.Scale(domain=[-7.5,0.5])
        ),
        tooltip=[
            "site",
            "natural_sequence_site",
            "wildtype",
            "effect",
            "n_effective",
            "entropy",
        ], 
    ).properties(
        width=300,
        height=300,
    )
    
    subplots.append(curr_subplot)

# Create final combined plot
natural_vs_func_effects = alt.hconcat(
    subplots[0],
    spacing=5,
    title="Functional effects vs natural variatio",
).configure_axis(
    grid=False,
    labelFontSize=16,
    titleFontSize=16,
    labelFontWeight="normal",
    titleFontWeight="normal",
).configure_title(
    fontSize=24,
).configure_view(
    stroke=None
)

natural_vs_func_effects

r correlation for all: 0.15


alt.HConcatChart(...)